In [254]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions

In [373]:
MacroTools.prettify(@macroexpand @model function smoothing(n_observations, noise_mean, noise_var)
    
    x_prior_mean = datavar(Float64)
    x_prior_var  = datavar(Float64)

    x_prior ~ NormalMeanVariance(x_prior_mean, x_prior_var) where { q = q(var)q(mean)q(out) }

    x = randomvar(n_observations)
    y = datavar(Float64, n_observations)

    x[1] = x_prior

    for i in 2:n_observations
        x[i] ~ x[i - 1] + 1.0
        y[i] ~ x[i] + (NormalMeanVariance(noise_mean, noise_var) where { q = q(var)q(mean)q(out) })
    end

    return x_prior_mean, x_prior_var, x, y
end)

:(function smoothing(n_observations, noise_mean, noise_var)
      finch = Model()
      x_prior_mean = ReactiveMP.datavar(finch, :x_prior_mean, Dirac{GraphPPL.ensure_type(Float64)})
      x_prior_var = ReactiveMP.datavar(finch, :x_prior_var, Dirac{GraphPPL.ensure_type(Float64)})
      (chinchilla, x_prior) = ReactiveMP.make_node(finch, NormalMeanVariance, ReactiveMP.AutoVar(:x_prior), ReactiveMP.as_variable(finch, Symbol("##arg#6798"), x_prior_mean), ReactiveMP.as_variable(finch, Symbol("##arg#6799"), x_prior_var); $(Expr(:(=), :factorisation, :(((1,), (2,), (3,))))))
      x = ReactiveMP.randomvar(finch, :x, n_observations)
      y = ReactiveMP.datavar(finch, :y, Dirac{GraphPPL.ensure_type(Float64)}, n_observations)
      x[1] = x_prior
      for i = 2:n_observations
          pheasant = ReactiveMP.make_node(finch, +, x[i], ReactiveMP.as_variable(finch, Symbol("##arg#6801"), x[i - 1]), ReactiveMP.as_variable(finch, Symbol("##arg#6802"), 1.0); )
          caterpillar = ReactiveMP.make_

In [390]:
@model function smoothing(n_observations, noise_mean, noise_var)
    
    x_prior_mean = datavar(Float64)
    x_prior_var  = datavar(Float64)

    x_prior ~ NormalMeanVariance(x_prior_mean, x_prior_var)

    x = randomvar(n_observations)
    y = datavar(Float64, n_observations)

    x[1] = x_prior

    for i in 2:n_observations
       x[i] ~ x[i - 1] + 1.0
       y[i] ~ x[i] + NormalMeanVariance(noise_mean, noise_var) where { q = FullFactorisation() }
    end

    return x_prior_mean, x_prior_var, x, y
end

LoadError: LoadError: Unknown option 'id = 1' for '+' node
in expression starting at In[390]:1

In [386]:
noise_real_mean = -10.0
noise_real_var  = 100.0

n = 500
data = map(d -> d + rand(Normal(noise_real_mean, sqrt(noise_real_var))), collect(1:n));

In [387]:
function inference(; data, x_prior)
    n = length(data)
    
    x_prior_mean, x_prior_var, x, y = smoothing(n, noise_real_mean, noise_real_var);

    subscriptions = Vector{Teardown}(undef, n)
    marginals = Vector{Marginal}(undef, n)
    
    for i in 1:n
         @inbounds subscriptions[i] = subscribe!(getmarginal(x[i]), (m) -> marginals[i] = m)
    end
    
    update!(x_prior_mean, mean(x_prior))
    update!(x_prior_var, var(x_prior))
    
    for i in 1:n
        @inbounds update!(y[i], data[i])
    end
    
    foreach(unsubscribe!, subscriptions)
    
    return marginals
end

inference (generic function with 1 method)

In [388]:
inference(
    data = data,
    x_prior = NormalMeanVariance(0.0, 10000.0)
)

500-element Array{Marginal,1}:
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=1.517198298550052, v=0.20039678563555818))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=2.517198298550052, v=0.20039678563555818))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=3.517198298550052, v=0.20039678563555818))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=4.517198298550051, v=0.20039678563555816))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=5.517198298550051, v=0.2003967856355582))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=6.517198298550051, v=0.20039678563555818))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=7.517198298550053, v=0.2003967856355582))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=8.517198298550055, v=0.20039678563555818))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=9.5171

In [345]:
@btime Symbol(join(ss, :_))

  672.757 ns (8 allocations: 352 bytes)


:a_a_a_a_a_a_a_a_a_a

In [89]:
@btime Symbol(ReactiveMP.with_separator(:_, ss)...)

  1.669 μs (42 allocations: 1.52 KiB)


:a_a_a_a_a_a_a_a_a_a

In [109]:
@btime datavar(:name, Float64, 100, 100)

  12.146 ms (248309 allocations: 10.48 MiB)


100×100 Array{DataVariable{Subject{Message{Float64},AsapScheduler,AsapScheduler},Float64},2}:
 DataVariable{Subject{Message{Float64},AsapScheduler,AsapScheduler},Float64}(:name_1_1, Subject(Message{Float64}, AsapScheduler), DataVariableProps(nothing, nothing))    …  DataVariable{Subject{Message{Float64},AsapScheduler,AsapScheduler},Float64}(:name_1_100, Subject(Message{Float64}, AsapScheduler), DataVariableProps(nothing, nothing))
 DataVariable{Subject{Message{Float64},AsapScheduler,AsapScheduler},Float64}(:name_2_1, Subject(Message{Float64}, AsapScheduler), DataVariableProps(nothing, nothing))       DataVariable{Subject{Message{Float64},AsapScheduler,AsapScheduler},Float64}(:name_2_100, Subject(Message{Float64}, AsapScheduler), DataVariableProps(nothing, nothing))
 DataVariable{Subject{Message{Float64},AsapScheduler,AsapScheduler},Float64}(:name_3_1, Subject(Message{Float64}, AsapScheduler), DataVariableProps(nothing, nothing))       DataVariable{Subject{Message{Float64},AsapScheduler

In [108]:
@btime datavar(:name, Float64, 100, 100)

  14.999 ms (250010 allocations: 10.22 MiB)


100×100 Array{DataVariable{Subject{Message{Float64},AsapScheduler,AsapScheduler},Float64},2}:
 DataVariable{Subject{Message{Float64},AsapScheduler,AsapScheduler},Float64}(:name_1_1, Subject(Message{Float64}, AsapScheduler), DataVariableProps(nothing, nothing))    …  DataVariable{Subject{Message{Float64},AsapScheduler,AsapScheduler},Float64}(:name_1_100, Subject(Message{Float64}, AsapScheduler), DataVariableProps(nothing, nothing))
 DataVariable{Subject{Message{Float64},AsapScheduler,AsapScheduler},Float64}(:name_2_1, Subject(Message{Float64}, AsapScheduler), DataVariableProps(nothing, nothing))       DataVariable{Subject{Message{Float64},AsapScheduler,AsapScheduler},Float64}(:name_2_100, Subject(Message{Float64}, AsapScheduler), DataVariableProps(nothing, nothing))
 DataVariable{Subject{Message{Float64},AsapScheduler,AsapScheduler},Float64}(:name_3_1, Subject(Message{Float64}, AsapScheduler), DataVariableProps(nothing, nothing))       DataVariable{Subject{Message{Float64},AsapScheduler

In [103]:
for index in CartesianIndices(axes(ss))
    println(join(index.I, :_))
end

1
2
3
4
5
6
7
8
9
10


In [99]:
r = CartesianIndices(axes(ss))[1]

CartesianIndex(1,)

In [104]:
?CartesianIndex

search: CartesianIndex CartesianIndices



```
CartesianIndex(i, j, k...)   -> I
CartesianIndex((i, j, k...)) -> I
```

Create a multidimensional index `I`, which can be used for indexing a multidimensional array `A`.  In particular, `A[I]` is equivalent to `A[i,j,k...]`.  One can freely mix integer and `CartesianIndex` indices; for example, `A[Ipre, i, Ipost]` (where `Ipre` and `Ipost` are `CartesianIndex` indices and `i` is an `Int`) can be a useful expression when writing algorithms that work along a single dimension of an array of arbitrary dimensionality.

A `CartesianIndex` is sometimes produced by [`eachindex`](@ref), and always when iterating with an explicit [`CartesianIndices`](@ref).

# Examples

```jldoctest
julia> A = reshape(Vector(1:16), (2, 2, 2, 2))
2×2×2×2 Array{Int64,4}:
[:, :, 1, 1] =
 1  3
 2  4

[:, :, 2, 1] =
 5  7
 6  8

[:, :, 1, 2] =
  9  11
 10  12

[:, :, 2, 2] =
 13  15
 14  16

julia> A[CartesianIndex((1, 1, 1, 1))]
1

julia> A[CartesianIndex((1, 1, 1, 2))]
9

julia> A[CartesianIndex((1, 1, 2, 1))]
5
```


In [102]:
dump(r.I)

Tuple{Int64}
  1: Int64 1


In [122]:
?@assert

```
@assert cond [text]
```

Throw an [`AssertionError`](@ref) if `cond` is `false`. Preferred syntax for writing assertions. Message `text` is optionally displayed upon assertion failure.

!!! warning
    An assert might be disabled at various optimization levels. Assert should therefore only be used as a debugging tool and not used for authentication verification (e.g., verifying passwords), nor should side effects needed for the function to work correctly be used inside of asserts.


# Examples

```jldoctest
julia> @assert iseven(3) "3 is an odd number!"
ERROR: AssertionError: 3 is an odd number!

julia> @assert isodd(3) "What even are numbers?"
```
